In [1]:
!pip install fasttext --no-cache-dir

In [21]:
import pandas as pd
import numpy as np
import fasttext
import nltk
import os
from os import path
from nltk.corpus import words
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def lemmatise(text):

    wn = nltk.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    word_filter = [token for token in tokens if token not in stopwords and token.isalpha()]
    lemma = [wn.lemmatize(token) for token in word_filter]

    return " ".join(lemma)

In [9]:
X_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [10]:
print(X_train.info())
print(X_train.shape)
print(X_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    25000 non-null  object
 1   label   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB
None
(25000, 2)
                                                text  label
0  bromwell high is a cartoon comedy it ran at th...      1
1  homelessness or houselessness as george carlin...      1
2  brilliant over acting by lesley ann warren bes...      1
3  this is easily the most underrated film inn th...      1
4  this is not the typical mel brooks film it was...      1


In [11]:
X_train['lemmas'] = X_train['text'].apply(lemmatise)
X_test['lemmas'] = X_test['text'].apply(lemmatise)

In [12]:
X_train['lemmas'].iloc[10]

'first read armistead maupins story taken human drama displayed gabriel one care love said given film version excellent story expected see past gloss hollywood writer armistead maupin director patrick stettner truly succeeded right amount restraint robin williams capture fragile essence gabriel let u see struggle issue trust personnel life jess world around donna introduced player drama reminded nothing ever seems smallest event change life irrevocably request review book written young man turn life changing event help gabriel find strength within carry move forward bad people avoid film say average american probably think robin williams serious role work please give movie chance robin williams touch darkness must find go better people like movie one hour photo stepped actor made another quality piece art oh forget believe bobby cannavale jess steal every scene leading man look screen presence hack opinion could carry movie right'

**FastText**

In [15]:
X_train['ft_label'] = X_train['label'].apply(lambda x: '__label__POSITIVE' if x == 1 else '__label__NEGATIVE')
X_test['ft_label'] = X_test['label'].apply(lambda x: '__label__POSITIVE' if x == 1 else '__label__NEGATIVE')
X_train['ft_text'] = X_train[['ft_label', 'lemmas']].apply(lambda x: ' '.join(x.map(str)), axis=1)
X_test['ft_text'] = X_test[['ft_label', 'lemmas']].apply(lambda x: ' '.join(x.map(str)), axis=1)

In [17]:
X_train['ft_text'][5]

'__label__POSITIVE comedic robin williams quirky insane robin williams recent thriller fame hybrid classic drama without dramatization mixed robin new love thriller thriller per se mystery suspense vehicle williams attempt locate sick boy keeper also starring sandra oh rory culkin suspense drama play pretty much like news report william character get close achieving goal must say highly entertained though movie fails teach guide inspect amuse felt like watching guy williams actually performing action third person perspective word felt real able subscribe premise story worth watch though definitely friday saturday night fare rate fiend'

In [28]:
X_train['ft_text'].to_csv('X_train.txt', header=False, index=False, sep='\t')
X_test['ft_text'].to_csv('X_test.txt', header=False, index=False, sep='\t')

In [30]:
model = fasttext.train_supervised(input="X_train.txt")

In [31]:
model.test("X_test.txt")

(25000, 0.8584, 0.8584)

In [42]:
model_cust = fasttext.train_supervised(input="X_train.txt", lr=1.0, epoch=25, wordNgrams=3)

In [43]:
model_cust.test("X_test.txt")

(25000, 0.884, 0.884)